# Taller 1
## [Introducción a los Sistemas Inteligentes 2021-1](https://fagonzalezo.github.io/iis-2021-1/)

--------------------

# Tallbots y Smallbots

Aima y Arya son mejores amigas, pero no solo son amigas. Son ingenieras de software y expertas en inteligencia artifical de la Unión Nacional de Algoritmos de Localización (UNAL). En su tiempo libre a Aima le gusta retar a Arya con distintos problemas y puzzles. 

Esta vez Aima decidio usar el problema de los Tallbots y Smallbots. Hay 2 Tallbots y 2 Smallbots, los bots quieren cruzar un rio y deben hacerlo en un bote, ya que si tocan el agua se dañarián. 

Pero hay un problema: El bote que tienen es muy pequeño para llevarlos a todos y solo pueden estar al mismo tiempo como máximo 1 Tallbot ó 2 Smallbots, por ejemplo si hubiera 2 Tallbots o 2 Smallbots y 1 Tallbot el bote se hundiría y los bots estarían tristes. Formalmente, el bote tiene capacidad de carga 2, los Smallbots tienen peso 1 y los Tallbots tienen peso 2. El bote no se puede mover si no lleva ningún bot. 

Sin embargo, Arya habiendo resuelto muchos problemas similares lo solucionó en menos de 1 minuto! Aima decidió darle una versión más díficil del problema a Arya: Dada una configuración inicial y final de bots Arya tiene que decirle a Aima la cantidad de pasos más pequeña y la correspondiente secuencia de pasos para llegar del estado inicial al estado final. En esta versión del problema puede haber una cantidad arbitraria de Tallbots y de Smallbots al igual que la capacidad del bote puede variar. Puede asumir que los movimientos de los bots son instantaneos, es decir, solo se cuenta como un paso de tiempo el movimiento de un lado a otro.

Arya esta completamente segura que puede escribir un programa que resuelva el problema general en menos de 10 minutos pero esta ocupada trabajando en un proyecto que le asigno el presidente de la UNAL. ¿Puedes ayudar a Arya a resolver este problema?

## 1. Defina una clase que represente el problema generalizado de Smallbots y Tallbots

Un estado del problema se puede representar usando una tupla de 3 posiciones donde el primer elemento representa el número de Tallbots al lado izquierdo, el segundo el númeor de Smallbots en el lado izquierdo y el último elemento de que lado se encuentra el bote. Por ejemplo, asumiendo que hay 2 Tallbots y 2 Smallbots, el siguiente vector:

```python
(1, 2, 0)
```

Representaria un estado en donde hay 1 Tallbot, 2 Smallbots y el bote en el lado izquierdo y un Tallbot en el lado derecho.


Su programa sera evaluado en distintos casos de prueba y debe soportar el siguiente formato de entrada y salida. (No cambiar nombre ni firma de los metodos)

**Entrada:**

**init** debe aceptar 4 parametros. **initial** y **goal** tuplas representando el estado inicial y final, **n** y **m** representando el número total de Tallbots y Smallbots correspondientemente y **c** la capacidad del bote. Todos los valores son números enteros.

Para cada estado **(tb, sb, l)**. **tb** es el número de Tallbots en el lado izquierdo, **sb** el número de Smallbots en el lado izquierdo y **l** el lado en que se encuentra el bote en el estado correspondiente, 0 significa que esta en la izquierda y 1 en la derecha.

Está garantizado que los valores cumplen con la siguientes condiciones:

**1 <= n + m <= 15**

**2 <= c <= 20**

**0 <= tb, sb <= n**

**0 <= li, lf <= 1**

**Salida**

La función `action` debe retornar todas las posibles acciones para un estado dado en una lista de tuplas. No importa el orden de la lista. El primer elemento de cada tupla representa cuantos Tallbots se van a mover y el segundo elemento cuantos Smallbots se van a mover.

La función `result` debe retornar el estado resultante de una acción sobre un estado. 

```python
p = TBSBProblem((1, 2, 0), (0, 0, 1), 1, 2, 2)
print(p.actions(p.initial))
>> [(0, 1), (0, 2), (1, 0)]
print(p.result(p.initial, (0, 2))
>> (1, 0, 1)
```

--------------------------------


```python
p = TBSBProblem((1, 2, 1), (0, 0, 1), 1, 2, 2)
print(p.actions(p.initial))
>> []
```

--------------------------------

```python
p = TBSBProblem((1, 2, 0), (0, 0, 1), 1, 2, 6)
print(p.actions(p.initial))
>> [(0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
print(p.result(p.initial, (1, 2))
>> (0, 0, 1)
print(p.is_goal(p.result(p.initial, (1, 2))))
>> True
```

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations, permutations

In [2]:
class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
        
class TBSBProblem(Problem):

    def __init__(self, initial=None, goal=None, n=0, m=0, c = 2):
        '''
        initial: tupla representando el estado inicial de la forma (tb, sb, side). 
                tb es el número de Tallbots en el lado izquierdo, sb el número 
                de Smallbots en el lado izquierdo y side el lado en que se encuentra 
                el bote en el estado correspondiente, 0 significa que esta en 
                la izquierda y 1 en la derecha.
        goal: tupla representando el estado objetivo, con la misma convención de
        initial.
        n: número total de Tallbots
        m: número total de Smallbots
        c: capacidad del bote
        '''
        tb, sb, l = initial
        t, s, lg = goal
        
        if((n+m < 1) or (n+m > 16)): raise Exception("Error")
        if((c < 2) or (c > 20)): raise Exception("Error")
        if((tb < 0) or (tb > n)): raise Exception("Error")
        if((sb < 0) or (sb > m)): raise Exception("Error")
        if(not (l in [0,1])): raise Exception("Error")
        if(not (lg in [0,1])): raise Exception("Error")
        
        self.initial = initial
        self.goal = goal
        self.n = n
        self.m = m
        self.c = c

    def actions(self, state):
        '''
        state: Tupla reperesentando un estado de la forma (tb, sb, side)

        Returns: [] Lista de tuplas representado las acciones posibles en el estado state,
        cada tupla con la forma (tb, sb) donde tb es el número de Tallbots que se van a mover
        y sb el número de Smallbots que se van a mover.
        '''
        t, s, x = state
        tr, sr = self.n - t, self.m - s # Right
        if(x == 1):
            t, s = tr, sr # Right    
            
        if(cmp(self.initial, state) and (x==1) and (tr == 0 and sr == 0)):
            return []
        c = self.c
        per =[(i,j) for i in range(t+1) for j in range(s+1)]
                
        return [(a, b) for (a, b) in per if ((a*2 + b <= c) & (not cmp((a,b),(0,0))))]


    def result(self, state, action):
        '''
        state: Tupla reperesentando un estado de la forma (tb, sb, side)
        action: Acción representada de la forma (tb, sb)

        Returns: Tupla representando el estado resultante al realizar la acción en el estado state.
        '''
        result = list()
        t, s, x = state
        tr, sr = self.n - t, self.m - s # Right
        if(cmp(self.initial, state) and (x==1) and (tr == 0 and sr == 0)):
            return []
        a, b = action
        
        if(x == 0):
            return (t-a, s-b, x^1)
        else:
            return (t+a, s+b, x^1)
    
    def is_goal(self, state):
        return cmp(self.goal, state)

In [3]:
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    # __lt__ == <, 
    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost

    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        
def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [4]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [5]:
def cmp(tp1, tp2):
    a = min(len(tp1), len(tp2))
    count = 0
    for i in range(a):
        if(tp1[i] == tp2[i]):
            count +=1    
    return count == a

In [6]:
a = max(1,2)
a

2

In [7]:
p = TBSBProblem((1, 2, 0), (0, 0, 1), 1, 2, 2)
print(p.actions(p.initial))

[(0, 1), (0, 2), (1, 0)]


In [8]:
print(p.result(p.initial, (0, 2)))

(1, 0, 1)


In [9]:
p = TBSBProblem((1, 2, 1), (0, 0, 1), 1, 2, 2)
print(p.actions(p.initial))

[]


In [10]:
p = TBSBProblem((1, 2, 0), (0, 0, 1), 1, 2, 6)
print(p.actions(p.initial))
print(p.result(p.initial, (1, 2)))
print(p.is_goal(p.result(p.initial, (1, 2))))

[(0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
(0, 0, 1)
True


In [11]:
cases = [[(1, 2, 0), (0, 0, 1), 1, 2, 2], [(1, 2, 1), (0, 0, 1), 1, 2, 2], [(1, 2, 0), (0, 0, 1), 1, 2, 6], [(2, 0, 1), (1, 1, 0), 2, 2, 2], [(2, 2, 0), (0, 0, 1), 2, 2, 2], [(4, 4, 0), (0, 0, 1), 4, 4, 4]]
exp_actions = [[(0, 1), (0, 2), (1, 0)], [], [(0, 1), (0, 2), (1, 0), (1, 1), (1, 2)], [(0, 1), (0, 2)], [(0, 1), (0, 2), (1, 0)], [(0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (2, 0)]]
p = TBSBProblem((2, 0, 1), (1, 1, 0), 2, 2, 2)
sorted(p.actions(p.initial)) 
# [(0, 1), (0, 2)]

[(0, 1), (0, 2)]

In [12]:
print(*cases[3])

(2, 0, 1) (1, 1, 0) 2 2 2


## 2. Defina una función que resuelva el problema general

Escriba una función que resuelva el problema de los Tallbots y Smallbots para una entrada igual del punto anterior. 

**Salida**
Debe retornar la longitud del camino más corto y la secuencia de pasos a seguir. Si hay varios soluciones retorne cualquiera y si no hay solución retorne -1.

**Ejemplos**

```python
>> print(solve((1, 2, 1), (0, 0, 1), 1, 2, 2))
>> -1
>> print(solve((1, 2, 0), (0, 0, 1), 1, 2, 2))
>> (5, [(0, 2), (0, 1), (1, 0), (0, 1), (0, 2)])
>> print(solve((2, 0, 1), (1, 1, 0), 2, 2, 2))
>> (5, [(0, 1), (1, 0), (0, 1), (0, 2), (0, 1)])
>> print(solve((2, 2, 0), (0, 0, 1), 2, 2, 2))
>> (9, [(0, 2), (0, 1), (1, 0), (0, 1), (0, 2), (0, 1), (1, 0), (0, 1), (0, 2)])
```
**Explicación:**
En el primer caso el bote empieza en la derecha y los bots en la izquierda, por lo que no pueden realizar ningún movimiento.

In [19]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

In [14]:
def solve(initial, goal, n, m, c):
    '''
    initial: tupla representando el estado inicial de la forma (tb, sb, side). 
            tb es el número de Tallbots en el lado izquierdo, sb el número 
            de Smallbots en el lado izquierdo y side el lado en que se encuentra 
            el bote en el estado correspondiente, 0 significa que esta en 
            la izquierda y 1 en la derecha.
    goal: tupla representando el estado objetivo, con la misma convención de
    initial.
    n: número total de Tallbots
    m: número total de Smallbots
    c: capacidad del bote

    Returns: tupla (x, [...]), donde x es el tamaño de la solucion más corta y [...] es la
    lista de acciones para solucionar el problema en x pasos. -1 si no existe solución.
    '''
    t,s, x = initial
    tr, sr = n - t, m - s # Right
    if((x==1) and (tr == 0 and sr == 0)):
            return -1
    problem = TBSBProblem(initial, goal, n, m, c)    
    node = breadth_first_search(problem)
    return (len(path_actions(node)), path_actions(node))

In [15]:
print(solve((1, 2, 1), (0, 0, 1), 1, 2, 2))

[]


In [16]:
print(solve((1, 2, 0), (0, 0, 1), 1, 2, 2))

(5, [(0, 2), (0, 1), (1, 0), (0, 1), (0, 2)])


In [20]:
print(solve((2, 0, 1), (1, 1, 0), 2, 2, 2))
# (5, [[(0, 1), (1, 0), (0, 1), (0, 2), (0, 1)]])

AttributeError: 'tuple' object has no attribute 'parent'

## 3. Defina una función que retorne todas las soluciones de longitud minima posibles al problema

**Entrada**
Igual que el punto anterior

**Salida**
Su programa debe retornar el tamaño de las soluciones y una lista de listas (o tuplas) que representen las soluciones, si no existe ninguna solución retornar -1

**Ejemplos**

```python
>> print(solve_all((2, 2, 0), (0, 0, 1), 2, 2, 2))
>> (9, [[(0, 2), (0, 1), (1, 0), (0, 1), (0, 2), (0, 1), (1, 0), (0, 1), (0, 2)]])
```
------------
```python
>> print(solve_all((4, 4, 0), (0, 0, 1), 4, 4, 4))
# Esta tiene 52 soluciones diferente. Se deben retornar todas.
>> (7,
 [[(0, 3), (0, 1), (1, 2), (0, 1), (2, 0), (0, 1), (1, 2)],
  [(0, 3), (0, 1), (2, 0), (0, 1), (2, 0), (0, 1), (0, 4)],
  [(0, 4), (0, 1), (2, 0), (0, 1), (1, 2), (0, 1), (1, 1)],
  [(0, 4), (0, 1), (2, 0), (0, 1), (1, 2), (1, 0), (2, 0)],
  [(0, 4), (0, 1), (2, 0), (0, 1), (2, 0), (0, 1), (0, 3)]
  ...
  ...
  ...])
```

In [18]:
def solve_all(initial, goal, n, m, c):
  '''
  initial: tupla representando el estado inicial de la forma (tb, sb, side). 
            tb es el número de Tallbots en el lado izquierdo, sb el número 
            de Smallbots en el lado izquierdo y side el lado en que se encuentra 
            el bote en el estado correspondiente, 0 significa que esta en 
            la izquierda y 1 en la derecha.
  goal: tupla representando el estado objetivo, con la misma convención de
  initial.
  n: número total de Tallbots
  m: número total de Smallbots
  c: capacidad del bote

  Returns: tupla (x, [[...], ...]) donde x es el tamaño de la solución más corta 
  y [[...], ...] es una lista de listas con todas las soluciones al problema de
   longitud x. Si no existe solución retornar -1.
  '''
  pass